<a href="https://colab.research.google.com/github/BlackeWhite/SSD_Modanet/blob/master/Prediction%26analysis_Instagram_Images/Prediction%26analysis_on_Instagram_Image_SSD300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library 

In [ ]:
#per salvare file di gorsse dimensioni sul drive 
!pip install httplib2==0.15.0
!pip install google-api-python-client==1.6
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install keras==2.2.3
!pip install tensorflow-gpu==1.15
import os.path
if not(os.path.isdir('keras_layers')):
  !git clone https://github.com/pierluigiferrari/ssd_keras.git
  !mv ./ssd_keras/* . 
  !rm -r ssd_keras

from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import SGD
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib
plt.ioff()

# Import Instagram Image and weight of SSD300

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Immagini Instagram estratte dagli hastag
Test = drive.CreateFile({'id': '1lloX-DOgQ7bG7iT0a0HMpfMNhOa-UvYi'}) 
Test.GetContentFile(Test['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Test['title'], Test['id']))
!unzip instagram_image.zip
#pesi SSD300 with Data augmentation
Weight = drive.CreateFile({'id': '1Ldszh6MHnGyQRrmZGn4-gln1SgQPGZ1v'}) 
Weight.GetContentFile(Weight['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Weight['title'], Weight['id']))

# Load the Model

In [ ]:
# Set the image size.
img_height = 300
img_width = 300

# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=13,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.5,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
weights_path = './ssd300_Modanet_data-aug_epoch-150_loss-5.3164_val_loss-5.1061.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=sgd, loss=ssd_loss.compute_loss)

# Creation of folders where save predicted images

In [ ]:
!mkdir pred_image
!mkdir "pred_image/#tods"
!mkdir "pred_image/#fendi"
!mkdir "pred_image/#dolcegabbana"
!mkdir "pred_image/#puma"
!mkdir "pred_image/#valentino"




# Prediction of Bounding box and analysis on Instagram Image

In [ ]:
import os

classes = ['background',
					 'bag', 'belt', 'boots', 'footwear',
					 'outer', 'dress', 'sunglasses', 'pants',
					 'top', 'shorts', 'skirt', 'headwear',
					 'scarf/tie']

lista_image = []
lista_hastag = []
lista_hastag = os.listdir("/content/instagram_image/")  #lista contenente i nomi delle 5 cartelle contenente le immagini dei diversi hastag 
for hast in lista_hastag:            #viene effettuato per ogni hastag 
	lista_image= os.listdir("/content/instagram_image/" + hast)
	lista_bag=0               #contatori abiti  
	lista_belt=0
	lista_boots=0
	lista_footwear=0
	lista_outer=0
	lista_dress=0
	lista_sunglasses=0
	lista_pants=0
	lista_top=0
	lista_shorts=0
	lista_skirt=0
	lista_headwear=0
	lista_scarfetie=0
	i=0 # conatore immagini predette 
	for item in lista_image:
		f = open(hast + ".csv", 'a') #file csv per salvare le predizioni delle immagini predette
		wrt = open(hast + "_stats.txt", 'w') 
		orig_images = [] # Store the images here.
		input_images = [] # Store resized versions of the images here.

		img_path = '/content/instagram_image/' + hast + "/" + item  

		orig_images.append(imread(img_path))
		img = image.load_img(img_path, target_size=(img_height, img_width))
		img = image.img_to_array(img) 
		input_images.append(img)
		input_images = np.array(input_images)
		y_pred = model.predict(input_images)
		confidence_threshold = 0.5
		y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

		np.set_printoptions(precision=2, suppress=True, linewidth=90)
		print("Predicted boxes for ",item,"\n")
		print('class   conf  xmin   ymin   xmax   ymax')
		print(y_pred_thresh[0])
		#stabiliamo una trashold basata sul numero di boundind box per ogni immagine predetta
	  #se il numero di boundind box è maggiore l'immagine viene processata e salvata 
	  #altrimenti viene scartata 

		if len(y_pred_thresh[0]) > 0:
			i=i+1
			f.write("Predicted boxes for image(300x300): " + str(item) + "\n")
			f.write('class,conf,xmin,ymin,xmax,ymax \n')
			for box in y_pred_thresh[0]:
				if round(box[0])==1:
					lista_bag+=1
				if round(box[0])==2:
					lista_belt+=1
				elif round(box[0])==3:
					lista_boots+=1
				elif round(box[0])==4:
					lista_footwear+=1 
				elif round(box[0])==5:
					lista_outer+=1                                                                            
				elif round(box[0])==6:
					lista_dress+=1
				elif round(box[0])==7:
					lista_sunglasses+=1   
				elif round(box[0])==8:
					lista_pants+=1  
				elif round(box[0])==9:
					lista_top+=1                                                          
				elif round(box[0])==10:
					lista_shorts+=1
				elif round(box[0])==11:
					lista_skirt+=1        
				elif round(box[0])==12:
					lista_headwear+=1   
				elif round(box[0])==13:
					lista_scarfetie+=1   
				f.write(str(box[0]) + "," + str(round(box[1],2)) + "," + str(round(box[2],2)) + "," + str(round(box[3],2)) + "," + str(round(box[4],2)) + "," + str(round(box[5],2)) +"\n" )
			colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
			plt.figure(figsize=(20,12))
			plt.axis('off')
			plt.imshow(orig_images[0])
			current_axis = plt.gca()
			f.write("\nPredicted boxes for original iamge: " + str(item) + "\n")
			f.write('class,conf,xmin,ymin,xmax,ymax \n')
			for box in y_pred_thresh[0]:
				# Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
				xmin = box[2] * orig_images[0].shape[1] / img_width
				ymin = box[3] * orig_images[0].shape[0] / img_height
				xmax = box[4] * orig_images[0].shape[1] / img_width
				ymax = box[5] * orig_images[0].shape[0] / img_height
				f.write(str(box[0]) + "," + str(round(box[1],2)) + "," + str(round(xmin,2)) + "," + str(round(ymin,2)) + "," + str(round(xmax,2)) + "," + str(round(ymax,2)) +"\n" )
				color = colors[int(box[0])]
				label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
				current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
				current_axis.text(xmin, ymin, label, size='x-large', color='black', bbox={'facecolor':color, 'alpha':1.0})  
				plt.savefig('./pred_image/' + hast + "/" + item, bbox_inches='tight', pad_inches=0) 
			plt.close()
			f.write("-------------------------------------- \n")
		del y_pred_thresh, y_pred
	wrt.write("N_images: " + str(i) + "\n")
	wrt.write("Bag: " + str(lista_bag)+ "\n")
	wrt.write("Belt: " + str(lista_belt)+ "\n")
	wrt.write("Boots: " + str(lista_boots)+ "\n")
	wrt.write("Footwear: " + str(lista_footwear)+ "\n")
	wrt.write("Outer: "+str(lista_outer)+ "\n")
	wrt.write("Dress: "+str(lista_dress)+ "\n")
	wrt.write("Sunglasses: "+str(lista_sunglasses)+ "\n")
	wrt.write("Pants: "+str(lista_pants)+ "\n")
	wrt.write("Top: "+str(lista_top)+ "\n")
	wrt.write("Shorts: "+str(lista_shorts)+ "\n")
	wrt.write("Skirt: "+str(lista_skirt)+ "\n")
	wrt.write("Headwear: "+str(lista_headwear)+ "\n")
	wrt.write("Scarf&tie: "+ str(lista_scarfetie)+ "\n")  
	wrt.close()
	f.close() 

# Save predict images and annotations

In [ ]:
!zip results.zip pred_image/*/* 
!cp results.zip "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#dolcegabbana_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#valentino_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#fendi_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#puma_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#tods_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#dolcegabbana.csv" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#valentino.csv" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#fendi.csv" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#puma.csv" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"
!cp "#tods.csv" "/content/drive/My Drive/SSD300_maggiore_di_1(almeno due bounding box)"


In [ ]:
!zip results.zip pred_image/*/* 
!cp results.zip "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#dolcegabbana_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#valentino_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#fendi_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#puma_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#tods_stats.txt" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#dolcegabbana.csv" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#valentino.csv" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#fendi.csv" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#puma.csv" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"
!cp "#tods.csv" "/content/drive/My Drive/SSD300_maggiore_di_0(almeno un bounding box)"